In [1]:
import os
from Bio import Entrez
import json

In [2]:
vazy2_path:str = "./../../../../testymolo/media/data/"
vazy2_list:list = [ 'Modulo', 'Profile', 'Organism', 'PolyProtein', 'Protein', 'Subseq', ]
vazy2:dict= {}

def Load(param:str) -> list:
    fileName:str = os.path.join(vazy2_path, param+'.temp.json')
    with open(fileName) as handle:
        return json.load(handle)
for tabName in vazy2_list:
    vazy2[tabName] = Load(tabName)

In [3]:
print(max([item['id'] for item in vazy2["Subseq"]]))

343


### trace

In [4]:
trace:dict = {}
for profile in vazy2['Profile']:
    k = profile['id']
    modId = profile['modulo']
    if modId.startswith('F'):
        trace[modId] =  []
        #filter
        for subSeq in vazy2['Subseq']:
            if subSeq['profile'] == k:
                trace[modId].append((subSeq['id'], subSeq['origin'], subSeq['profile'], subSeq['start'], subSeq['end']))
trace
with open("./vazi2_mod_trace.json", 'w') as out :
    out.write(json.dumps(trace))

# Extract profile fasta

In [11]:
for mod, SubSeqs in trace.items():
    fasta:list = []
    for ss in SubSeqs:
        #filter
        for item in vazy2['Protein']:
            if item['id'] == ss[1]:
                header:str = f">{mod}.{ss[1]}.{ss[0]}.{item['definition']}.[{ss[-2]}-{ss[-1]}]\n"
                sequence:str = item['sequence'][ss[-2]-1:ss[-1]-1]+'\n'
                fasta.append(header+sequence)
                break
    #endfor
    with open(f"./fasta/{mod}.fasta", 'w') as out:
        out.write(''.join(fasta))
        

### Alignment fasta

In [12]:
from Bio.Align.Applications import MuscleCommandline
import subprocess 

for file in os.listdir("./fasta"):
    #print(file)
    inFile:str = os.path.join("./fasta", file)
    out:str = os.path.join('./aln', file.split(".fasta")[0]+'.aln'+'.fasta')
    
    command:str = f"muscle -in {inFile} -out {out} -quiet"
    print(command)  # adding "-quiet" option ... doen't work in v3.8 ?! (but does in v5.)
    subprocess.Popen(command.split(' '), stdout=subprocess.DEVNULL)  # command must be a list of words



muscle -in ./fasta/F191.fasta -out ./aln/F191.aln.fasta -quiet
muscle -in ./fasta/F252.fasta -out ./aln/F252.aln.fasta -quiet
muscle -in ./fasta/F200.fasta -out ./aln/F200.aln.fasta -quiet
muscle -in ./fasta/F189.fasta -out ./aln/F189.aln.fasta -quiet
muscle -in ./fasta/F247.fasta -out ./aln/F247.aln.fasta -quiet
muscle -in ./fasta/F197.fasta -out ./aln/F197.aln.fasta -quiet
muscle -in ./fasta/F253.fasta -out ./aln/F253.aln.fasta -quiet
muscle -in ./fasta/F204.fasta -out ./aln/F204.aln.fasta -quiet
muscle -in ./fasta/F248.fasta -out ./aln/F248.aln.fasta -quiet
muscle -in ./fasta/F192.fasta -out ./aln/F192.aln.fasta -quiet
muscle -in ./fasta/F233.fasta -out ./aln/F233.aln.fasta -quiet
muscle -in ./fasta/F40.fasta -out ./aln/F40.aln.fasta -quiet
muscle -in ./fasta/F190.fasta -out ./aln/F190.aln.fasta -quiet
muscle -in ./fasta/F185.fasta -out ./aln/F185.aln.fasta -quiet
muscle -in ./fasta/F184.fasta -out ./aln/F184.aln.fasta -quiet
muscle -in ./fasta/F251.fasta -out ./aln/F251.aln.fasta -

### transfer  

* *foreach PP -> Map les nsp  =>  vazy2["PolyProtein"]*

* *foreach subseq -> find the nsp that fits best* \
    1. indices comparison (auto)

In [6]:
def filter(tab:list, **kwargs) -> list:
    res:list = []
    for item in tab:
        params = [item[key] == value for key, value in kwargs.items()]
        #print(params)
        if all(params):
            res.append(item)
            #print(item)
            pass
        elif any(params):
            #print(item)
            pass
    return res

In [5]:
def interval(val:int, t:int=1):
    return range(val-t,val+t)

def indices_comparison(hits:list, **positions):
    result:list = []
    for hit in hits:
        bools = [hit[key] in interval(value) for key, value in positions.items()]
        #bools = [positions['start'] >= hit['start']-1 , positions['end'] <= hit['end']+1]
        #print(bools)
        if all(bools):
            result.append(hit)
        elif all([positions['start'] >= hit['start']-1 , positions['end'] <= hit['end']+1]):
            result.append(hit)
    return result

In [7]:
notr = {
     #'F187': [368, 56, 7, 4123, 4232] ,
     #'F185': [198, 31, 9, 3924, 4015] ,
     'F196': [201, 31, 12, 2868, 3333] ,
     }
#
for key, value in notr.items():
    pps = filter(vazy2['PolyProtein'], PP=value[1])
    ps = filter(vazy2['PolyProtein'], protein=value[1])
    if len(pps) > 0:
        print(len(pps))
        hits = indices_comparison(pps, start=value[3], end=value[4])
        if len(hits) > 0:
             print(*hits, sep='\n')
        else:
             print("pps: no hits")
             print(*pps, sep='\n')

    elif len(ps) > 0:
        print(len(ps))
        hits = indices_comparison(ps, start=value[3], end=value[4])
        if len(hits) > 0:
             print(*hits, sep='\n')
        else:
             print("ps: no hits")
             print(*ps, sep='\n')
    break

15
pps: no hits
{'PP': 31, 'protein': 153, 'index': 1, 'start': 1, 'end': 247}
{'PP': 31, 'protein': 154, 'index': 2, 'start': 248, 'end': 832}
{'PP': 31, 'protein': 155, 'index': 3, 'start': 833, 'end': 3141}
{'PP': 31, 'protein': 377, 'index': 4, 'start': 3142, 'end': 3333}
{'PP': 31, 'protein': 156, 'index': 5, 'start': 3334, 'end': 3636}
{'PP': 31, 'protein': 157, 'index': 6, 'start': 3637, 'end': 3923}
{'PP': 31, 'protein': 158, 'index': 7, 'start': 3924, 'end': 4012}
{'PP': 31, 'protein': 159, 'index': 8, 'start': 4013, 'end': 4209}
{'PP': 31, 'protein': 160, 'index': 9, 'start': 4210, 'end': 4319}
{'PP': 31, 'protein': 161, 'index': 10, 'start': 4320, 'end': 4456}
{'PP': 31, 'protein': 162, 'index': 12, 'start': 4457, 'end': 5384}
{'PP': 31, 'protein': 163, 'index': 13, 'start': 5385, 'end': 5984}
{'PP': 31, 'protein': 164, 'index': 14, 'start': 5985, 'end': 6505}
{'PP': 31, 'protein': 165, 'index': 15, 'start': 6506, 'end': 6879}
{'PP': 31, 'protein': 166, 'index': 16, 'start':

In [59]:

vazi2_mod_trace:dict = {}

with open("./vazi2_mod_trace.json") as handle :
    vazi2_mod_trace = json.load(handle)

all_pp1a:list = [2, 24, 32, 43, 69, 269, 329, 341, 343]
all_PP:list = [ item['PP'] for item in vazy2['PolyProtein'] if item['PP'] not in all_pp1a]

In [60]:
trace_transfer:dict = {}
trace_no_nsp:dict = {}
trace_mlt_hit:dict = {}
trace_done:dict = {}
trace_no_transfer:dict = {}

for modId, SubSeqs in vazi2_mod_trace.items():
    #print("modulo :", modId, sep='\t')
    #trace_transfer[modId] = []
    for sseq in SubSeqs:
        
        #--> Is PolyProtein
        items_filtered:list = filter(vazy2['PolyProtein'], PP=sseq[1])
        #--> filter management
        
        if len(items_filtered) < 1:
            items_filtered = filter(vazy2['PolyProtein'], protein=sseq[1])
            if len(items_filtered) < 1:
                if not (modId in trace_no_nsp):
                    trace_no_nsp[modId] = []
                #sseq.append(False)
                trace_no_nsp[modId].append(sseq)    
        #print("no NSP for:", modId, sseq)
        #break

        hits:list = indices_comparison(items_filtered, start=sseq[3], end=sseq[4])
        #print(*hits, sep='\n')

        # hit management
        if len(hits) > 1:
            """if not modId in trace_mlt_hit:
                trace_mlt_hit[modId] = []
            sseq.append(False)
            trace_mlt_hit[modId].append(sseq)
            #print("multiple hits", modId, sseq)"""
            continue
        elif len(hits) < 1:
            #print("no hit:", modId, sseq)
            if not sseq[1] in all_pp1a:

                #--> already done ?
                if (sseq[1] not in all_PP):
                    if not (modId in trace_done):
                        trace_done[modId] = []
                    trace_done[modId].append(sseq)
                    continue

                if not (modId in trace_no_transfer):
                    trace_no_transfer[modId] = []
                print("'"+modId+"':", sseq, ",")
                trace_no_transfer[modId].append(sseq)
            continue
        ##
        else:
            if not (modId in trace_transfer):
                trace_transfer[modId] = []
            start = sseq[3] - hits[0]['start'] +1
            end = sseq[4] - sseq[3] +1
            trace_transfer[modId].append((sseq[0], hits[0]["protein"], sseq[2], start, end))
        
            

        #break
        #endfor

    #break 
    #endfor



'F187': [368, 56, 7, 4123, 4232] ,
'F185': [198, 31, 9, 3924, 4015] ,
'F196': [201, 31, 12, 2868, 3333] ,


In [62]:
with open("./trace_transfer.json", 'w') as out :
    out.write(json.dumps(trace_transfer))
    
with open("./trace_no_nsp.json", 'w') as out :
    out.write(json.dumps(trace_no_nsp))
    
#with open("./trace_mlt_hit.json", 'w') as out :
#    out.write(json.dumps(trace_mlt_hit))
    
with open("./trace_done.json", 'w') as out :
    out.write(json.dumps(trace_done))


with open("./trace_no_transfer.json", 'w') as out :
    out.write(json.dumps(trace_no_transfer))


### alignment profile .vs. PP

In [15]:
def profile_alignment(modId, sseq):
    # profile
    fasta_path:str = "./aln/"+modId+'.aln.fasta'
    # input protein
    #with open("./sequence_input.fasta", 'w') as inFile:
    for protein in vazy2['Protein']:
        if protein['id'] == sseq[1]:
            text:str = ">"+protein['header']+"\n"+protein['sequence']
            #inFile.write('>'+protein['header']+'\n'+protein['sequence'])
            #command:str = f"muscle -in {filepath} -out {out_filepath} -quiet"
            return text



In [16]:
print(profile_alignment("F188", [23, 2, 6, 3784, 3928]))

>ORF1a_polyprotein__(@474)
MASSLKQGVSPKPRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEECDTNSECEEEDEDTKVLALIQDPASIKYPLPLDEDYSVYNGCIVHKDALDVVNLPSGEETFVVNNCFEGAVKPLPQKVVDVLGDWGEAVDAQEQLCQQEPLQHTFEEPVENSTGSSKTMTEQVVVEDQELPVVEQDQDVVVYTPTDLEVAKETAEEVDEFILIFAVPKEEV

# Comparator

*impossible cuz not same identification system*

In [ ]:
import json, os 

vazi1_mod_trace:dict = {}
vazi2_mod_trace:dict = {}

with open("./vazi1_mod_trace.json") as handle :
    vazi1_mod_trace = json.load(handle)
with open("./vazi2_mod_trace.json") as handle :
    vazi2_mod_trace = json.load(handle)

# 

### compare advancement

In [1]:
import json, os 

vazi2_mod_trace:dict = {}
with open("./vazi2_mod_trace.json") as handle :
    vazi2_mod_trace = json.load(handle)

trace_transfer:dict = {}
with open("./trace_transfer.json") as handle:
    trace_transfer = json.load(handle)

trace_no_transfer:dict = {}
with open("./trace_no_transfer.json") as handle :
    trace_no_transfer = json.load(handle)


In [2]:
SubSeqs_id:list = []
all_pp1a:list = [2, 24, 32, 43, 69, 269, 329, 341, 343]

for modId, SubSeqs in vazi2_mod_trace.items():
    for sseq in SubSeqs:   
        if (not sseq[0] in SubSeqs_id) and (not sseq[1] in all_pp1a):
            SubSeqs_id.append(sseq[0])

for modId, SubSeqs in trace_transfer.items():
    for sseq in SubSeqs:   
        if sseq[0] in SubSeqs_id:
            SubSeqs_id.remove(sseq[0])

for modId, SubSeqs in trace_no_transfer.items():
    for sseq in SubSeqs:   
        if sseq[0] in SubSeqs_id:
            SubSeqs_id.remove(sseq[0])

SubSeqs_id.sort()

In [3]:
print(len(SubSeqs_id))
print(SubSeqs_id)

20
[1, 5, 57, 59, 186, 188, 231, 250, 252, 253, 313, 315, 358, 360, 385, 386, 403, 405, 474, 476]


# Parsing .m8 into graph
### Step 1: Parse the .m8 file